# Autograding for Python/Jupyter/Excel using Otter Grader & GitHub Classroom or Blackboard

Please see the readme for initial installation and setup instructions prior to running the notebook [here](https://github.com/jkuruzovich/otter_helper/blob/master/README.md). 

This notebook will allow you to grade a Python assignment and upload the results to Blackboard.


In [1]:
%load_ext autoreload
%autoreload 2
%matplotlib inline

### Set Course and Assignment
 Update set the `course` and `assignment_id` variables to be consistent with the `config.yml` file. 
 
 To run the sample assignments, just keep `course` set to `sample-class` and select `blackboard` or `github-classroom` for the `assignment_id`.

In [2]:
#set to what you want to grade
course = 'sample-class' #Sample assignments. 
assignment_id = 'github-classroom'  #Choose 'blackboard' or 'github-classroom' for samples. 


### Set and Load the Configuration
This loads the configuration from the `/config/course/config.yml` file.  

In [3]:
from pathlib import Path
import configparser, sys, os, importlib
import pandas as pd
 
cwd_dir = Path.cwd() #For running locally
base_dir = cwd_dir.parent #For running locally
#base_dir='/content/drive/My Drive/autograding'
modules_path = base_dir / 'modules' 
sys.path.append('../modules') # Not sure why appending the mudles path didn't work. 

#Load the autograding library
import autograde as ag
importlib.reload(ag)
cf = ag.get_config(course, assignment_id, base_dir)
cf

{'class_name': 'Sample Class',
 'class_id': 'sample-class',
 'message_complete': 'Your submission was successfully received and graded.<br>',
 'message_incomplete': 'If you get this message it is because you did not submit assignment. Please see the TA.<br>',
 'num_containers': 4,
 'ignore': ['.ipynb_checkpoints', '.DS_Store'],
 'requirements': 'config/sample-class/requirements.txt',
 'roster': 'config/sample-class/roster.xlsx',
 'assignments': {'blackboard': {'name': 'Sample assignment',
   'type': 'bb',
   'extension': '.ipynb',
   'tests_path': 'config/sample-class/blackboard/hidden-tests',
   'requirements': 'config/sample-class/requirements.txt',
   'seed': 42,
   'bb_column': 'blackboard_grade_test',
   'files': ['config/files/test_data_a.csv', 'config/files/test_data_b.csv']},
  'github-classroom': {'name': 'Sample assignment2',
   'type': 'gc',
   'extension': '.ipynb',
   'tests_path': 'config/sample-class/github-classroom/hidden-tests',
   'requirements': 'config/sample-class

### Prepare Grading
The will copy the assignent files either from the Blackboard or the GitHub classroom files in the `/assignments/course/assignment` folder to the `/tmp/` folder. It will also generate the needed `meta.json` file which maps the identifier (GitHub/Student ID) to the submission file. 

*For BlackBoard*
This makes the assumption that for files collected via Blackboard, each assignment file will be in the form `assignment_identifier_submissiondate`. The identifier is pulled out.  If your pattern is different, minor changes might be needed. [TBD, include a pattern in config.]

*For Github*
This makes the assumption that for files collected via Github Classroom, each assignment is in a different directory, where the directory name is the student's GitHub id. This GitHub id is used as the identifier for grading and matched with the student id after grading. 

The `cleaup=True` option just deletes the tmp folder before starting and is recommended.  

In [4]:
files=ag.prepare_grade(cf,  cleanup=True)  
files

[{'identifier': 'demo-fails1Hidden',
  'filename': 'demo-fails1Hidden_notebook.ipynb'},
 {'identifier': 'demo-fails2', 'filename': 'demo-fails2_notebook.ipynb'},
 {'identifier': 'demo-passesAll', 'filename': 'demo-passesAll_notebook.ipynb'},
 {'identifier': 'demo-fails3', 'filename': 'demo-fails3_notebook.ipynb'},
 {'identifier': 'demo-fails1', 'filename': 'demo-fails1_notebook.ipynb'},
 {'identifier': 'demo-fails3Hidden',
  'filename': 'demo-fails3Hidden_notebook.ipynb'},
 {'identifier': 'demo-fails2Hidden',
  'filename': 'demo-fails2Hidden_notebook.ipynb'}]

### Running the Grader 

This will launch several Docker containers (the number specified by the `config.yml` file, grade the assignment, and aggregate the assocated grades in the `./tmp/final_grades.csv`.

In [5]:
!cd ../tmp && otter -j meta.json -t tests --image otter-helper:latest -v

Found JSON metadata...
Launching docker containers...
Launched container f1d460f2c5a0...
Launched container 38f8a1b0ac6b...
Launched container 21511bdaccd6...
Launched container 1bddc21b9216...
Installing requirements in container f1d460f2c5a0...
Installing requirements in container 38f8a1b0ac6b...
Installing requirements in container 21511bdaccd6...
Installing requirements in container 1bddc21b9216...
Grading notebooks in container 1bddc21b9216...
Grading notebooks in container f1d460f2c5a0...
Grading notebooks in container 38f8a1b0ac6b...
Grading notebooks in container 21511bdaccd6...
Copying grades from container f1d460f2c5a0...
Stopping container f1d460f2c5a0...
Copying grades from container 1bddc21b9216...
Copying grades from container 38f8a1b0ac6b...
Stopping container 1bddc21b9216...
Copying grades from container 21511bdaccd6...
Stopping container 38f8a1b0ac6b...
Stopping container 21511bdaccd6...
Combining grades and saving...


### Review the Grades
No you have graded all the students there is an output file in `./tmp/final_grades.csv`.  If you have added your solution to this, it is a good practice to double check that your solution passed all of the associate tests. 

If you find that nearly all students missed a test, it may also signal an issue with the test. You can always adjust the tests and then re-run the above grading. 


In [6]:
grades = pd.read_csv(cf['grade_file'])
grades

,identifier,file,q1,q1H,q2,q2H,q3,q3H,total,possible
0,demo-passesAll,demo-passesAll_notebook.ipynb,1.0,2.0,1.0,1.0,1.0,2.0,8.0,8
1,demo-fails3,demo-fails3_notebook.ipynb,1.0,2.0,1.0,1.0,0.0,0.0,5.0,8
2,demo-fails2Hidden,demo-fails2Hidden_notebook.ipynb,1.0,2.0,1.0,0.0,1.0,2.0,7.0,8
3,demo-fails1,demo-fails1_notebook.ipynb,0.0,0.0,1.0,1.0,1.0,2.0,5.0,8
4,demo-fails2,demo-fails2_notebook.ipynb,1.0,2.0,0.0,0.0,1.0,2.0,6.0,8
5,demo-fails1Hidden,demo-fails1Hidden_notebook.ipynb,1.0,0.0,1.0,1.0,1.0,2.0,6.0,8
6,demo-fails3Hidden,demo-fails3Hidden_notebook.ipynb,1.0,2.0,1.0,1.0,1.0,0.0,6.0,8


### Archive `tmp` Directory and Generate the Upload to Blackboard File

This will archive the `/tmp` directory to the `/output/course/assignment` directory so that you have a persistent copy of the final version.   


This will loop through the students in your roster file (Blackboard tab) and match it with any graded submissions.  For GitHub classroom it will also match the github ID with the student ID. It will then generate the `/output/course/assignment/upload.csv` file which is ready to be uploaded to blackboard. Scores on each test are included in the `Feedback to Learner` column along with the message included in the `config.yml` file.

**It is important to make sure that your `roster.xslx` file is up to date. If a student entered the class late and isn't on the roster, that student's grade won't be updated.**

**It is important to make sure that the `bb_column` in the `config.yml` file matches the column in Blackboard when you download for offline grading.**



In [7]:
#Aggregate JSON files for manual grading. 
blackboard=ag.prepare_blackboard_upload(cf, archive=True)
blackboard

complete: 6 
Incomplete: 4 
Total: 10
Archiving files in 


,Last Name,First Name,Username,Student ID,Last Access,Availability,blackboard_grade_test,Grading Notes,Notes Format,Feedback to Learner,Feedback Format
0,Name,Name,fails3,fails3,2020-08-01 22:08:00,Yes,5.0,NaN,NaN,Your submission was successfully received and ...,HTML
1,Name,Name,passesAll,passesAll,2020-08-01 22:08:00,Yes,8.0,NaN,NaN,Your submission was successfully received and ...,HTML
2,Name,Name,fails2Hidden,fails2Hidden,2020-08-01 22:08:00,Yes,7.0,NaN,NaN,Your submission was successfully received and ...,HTML
3,Name,Name,fails3Hidden,fails3Hidden,2020-08-01 22:08:00,Yes,6.0,NaN,NaN,Your submission was successfully received and ...,HTML
4,Name,Name,fails2,fails2,2020-08-01 22:08:00,Yes,6.0,NaN,NaN,Your submission was successfully received and ...,HTML
5,Name,Name,fails1,fails1,2020-08-01 22:08:00,Yes,5.0,NaN,NaN,Your submission was successfully received and ...,HTML
6,Name,Name,fails4,fails4,2020-08-01 22:08:00,Yes,0.0,NaN,NaN,If you get this message it is because you did ...,NaN
7,Name,Name,fails3ab,fails3ab,2020-08-01 22:08:00,Yes,0.0,NaN,NaN,If you get this message it is because you did ...,NaN


## Upload to Blackboard File 

TBD: Flag student submissions which aren't in the roster file.

Upload the `/output/course/assignment/upload.csv` and you are done. 